In [1]:
# Importing required libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
import os

2024-06-21 16:01:58.798711: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-21 16:01:59.653164: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-06-21 16:01:59.653289: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-06-

In [2]:
# Function to load and preprocess data based on the test case
def load_and_preprocess_data(file_path, test_case):
    df = pd.read_csv(file_path)
    df['datetime'] = pd.to_datetime(df['date/time'], errors='coerce')
    
    if test_case == '4.csv':
        # Calculate the start and end times for trimming
        train_start_time = df['datetime'].min() + pd.Timedelta(minutes=1.5)
        train_end_time = df['datetime'].max() - pd.Timedelta(minutes=1.5)
        
        # Filter the DataFrame to exclude the first and last 1.5 minutes of data
        df = df[(df['datetime'] > train_start_time) & (df['datetime'] < train_end_time)]
        
        # Get the total number of rows in the filtered DataFrame
        total_rows = len(df)
        
        # Slice the DataFrame in half and use the second half
        df = df.iloc[total_rows//2:]
        
    elif test_case == '6.csv':
        # Remove the first 2 minutes and the last 1 minute
        train_start_time = df['datetime'].min() + pd.Timedelta(minutes=2)
        train_end_time = df['datetime'].max() - pd.Timedelta(minutes=1)
        
        # Filter the DataFrame to exclude the first 2 minutes and the last 1 minute of data
        df = df[(df['datetime'] > train_start_time) & (df['datetime'] < train_end_time)]
        
    elif test_case == '7.csv':
        # Remove the first and last minute
        train_start_time = df['datetime'].min() + pd.Timedelta(minutes=1)
        train_end_time = df['datetime'].max() - pd.Timedelta(minutes=1)
        
        # Filter the DataFrame to exclude the first and last minute of data
        df = df[(df['datetime'] > train_start_time) & (df['datetime'] < train_end_time)]
        
    else:
        raise ValueError("Unknown test case")
    
    return df


In [3]:
# Function to create sequences
def create_sequences(data, sequence_length=1600):
    sequences = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
    return np.array(sequences)

In [4]:
# Function to run the test
def run_test(model_path, test_file, sequence_length):
    # Load the model
    model = load_model(model_path)
    
    # Load and preprocess the data
    test_df = load_and_preprocess_data(test_file, test_file.split('/')[-1])
    
    # Define features and targets
    features = ['gyro_x', 'gyro_y', 'gyro_z', 'accel_x', 'accel_y', 'accel_z', 'x_cam', 'y_cam', 'z_cam']
    targets = ['x', 'y']
    
    # Scale the features
    scaler = StandardScaler()
    test_X_scaled = scaler.fit_transform(test_df[features])
    
    # Extract target values
    test_y = test_df[targets].values
    
    # Convert the data into sequences
    X_test = create_sequences(test_X_scaled, sequence_length)
    y_test = test_y[sequence_length:]
    time_test = test_df['datetime'].iloc[sequence_length:].reset_index(drop=True)

    # Predict using the test data
    y_pred = model.predict(X_test)
    
    # Create a dataframe to store the results
    results_df = pd.DataFrame({
        'index': range(len(y_test)),
        'time': time_test,
        'true_x': y_test[:, 0],
        'true_y': y_test[:, 1],
        'predicted_x': y_pred[:, 0],
        'predicted_y': y_pred[:, 1],
    })
    
    return results_df

In [5]:
networks_to_test = [
    ('trajectory_model10.h5', 'own/data/6.csv', 1600),
    ('trajectory_model10.h5', 'own/data/4.csv', 1600),
    ('trajectory_model10.h5', 'own/data/7.csv', 1600),
    ('trajectory_model9.h5', 'own/data/6.csv', 800),
    ('trajectory_model9.h5', 'own/data/4.csv', 800),
    ('trajectory_model9.h5', 'own/data/7.csv', 800),
    ('trajectory_model9i.h5', 'own/data-int/6.csv', 800),
    ('trajectory_model9i.h5', 'own/data-int/4.csv', 800),
    ('trajectory_model9i.h5', 'own/data-int/7.csv', 800),
    ('trajectory_model10i.h5', 'own/data-int/4.csv', 1600),
    ('trajectory_model10i.h5', 'own/data-int/6.csv', 1600),
    ('trajectory_model10i.h5', 'own/data-int/7.csv', 1600),
    ('trajectory_model11.h5', 'own/data-int/4.csv', 3200),
    ('trajectory_model11.h5', 'own/data-int/6.csv', 3200),
    ('trajectory_model11.h5', 'own/data-int/7.csv', 3200),
    # add more tests here the format is |model name|test case|sequence length|
]

for network in networks_to_test:
    model_path, test_file, sequence_length = network
    model_name = os.path.basename(model_path).split('.')[0]
    test_number = os.path.basename(test_file).split('.')[0]
    results_filename = f'{model_name}-{test_number}.csv'
    results_dir = 'own/results'
    results_path = os.path.join(results_dir, results_filename)

    # Check if the results file already exists
    if os.path.exists(results_path):
        print(f'Results file {results_path} already exists. Skipping test.')
        continue

    # Run the test and get the results
    results_df = run_test(model_path, test_file, sequence_length)

    # Check if results_df is None
    if results_df is None:
        print(f'Test for {model_path} with {test_file} returned no results. Skipping save.')
        continue

    # Create the results directory if it doesn't exist
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
    
    # Save the results to a CSV file
    results_df.to_csv(results_path, index=False)
    print(f'Results saved to {results_path}')

Results file own/results/trajectory_model10-6.csv already exists. Skipping test.
Results file own/results/trajectory_model10-4.csv already exists. Skipping test.
Results file own/results/trajectory_model10-7.csv already exists. Skipping test.
Results file own/results/trajectory_model9-6.csv already exists. Skipping test.
Results file own/results/trajectory_model9-4.csv already exists. Skipping test.
Results file own/results/trajectory_model9-7.csv already exists. Skipping test.
Results file own/results/trajectory_model9i-6.csv already exists. Skipping test.
Results file own/results/trajectory_model9i-4.csv already exists. Skipping test.
Results file own/results/trajectory_model9i-7.csv already exists. Skipping test.
Results file own/results/trajectory_model10i-4.csv already exists. Skipping test.
Results file own/results/trajectory_model10i-6.csv already exists. Skipping test.
Results file own/results/trajectory_model10i-7.csv already exists. Skipping test.
Results file own/results/tra